# Recommendations Systems
## Assignment 3:  Neural Collaborative Filtering

**By:**  
Group 16

<br><br>

**The goal of this assignment is to:**
- Understand the concept of recommendations based on implicit data which is very common in real life.
- Understand how DL components can be used to implement a collaborative filtering & hybrid approach recommenders.
- Understand pros&cons comparing to other recommender system approaches.
- Practice recommender system training and evaluation.

**Instructions:**
- Students will form teams of two people each, and submit a single homework for each team.
- The same score for the homework will be given to each member of the team.
- Your solution in the form of an Jupyter notebook file (with extension ipynb).
- Images/Graphs/Tables should be submitted inside the notebook.
- The notebook should be runnable and properly documented. 
- Please answer all the questions and include all your code.
- English only.

**Submission:**
- Submission of the homework will be done via Moodle by uploading a link to google colab.
- The homwork needs to be entirely in English.
- The deadline for submission is on Moodle.

**Requirements:**  
- Python 3.6+ should be used. 
- You may use Torch/Keras/TF packeges.
- You should implement the recommender system by yourself using only basic Python libraries (such as numpy).

**LINKS:**
- <a href='https://github.com/hexiangnan/neural_collaborative_filtering/tree/master/Data'>Dataset</a>
- <a href='https://github.com/hexiangnan/neural_collaborative_filtering'>Repository</a>
- <a href='https://towardsdatascience.com/paper-review-neural-collaborative-filtering-explanation-implementation-ea3e031b7f96'>Blog Post Review</a>
<br>

**Google <a href='https://colab.research.google.com/'>Colaboratory</a>**  
        
    This is a great academic tool for students. Instead of installing and running "everything" on your Laptop - which probably will take you a lot of time - you can use Google Colab.  
    Basically, you can use it for all your Python needs.  

**PyTorch <a href='https://pytorch.org/tutorials/beginner/basics/intro.html'>Tutorials</a>**   
    
    Just follow steps 0-7 and you will have the basics skills to understand, build, and run DL recommender models. 

**Keras Kaggle's <a href='https://www.kaggle.com/learn/intro-to-deep-learning'>intro-to-deep-learning</a>**  
    
    This will give you a quick idea of what DL is, and how to utilize it.  
    They're using TensorFlow, while in our MLDL program we're using PyTorch.  




**Grading:**

- Q1 - 20 points - Dataset Preparation
- Q2 - 50 points - Neural Collaborative Filtering
- Q3 - 30 points - Loss Function

`Total: 100`

<br><br><br>

**Prerequisites**

In [ ]:
%pip install torch torchvision --quiet

**Imports**

In [ ]:
# basic
import os 
import sys
import math
import heapq
import argparse
from time import time
import multiprocessing

# general
import warnings
import numpy as np
import scipy as sp
import pandas as pd
import scipy.sparse as sp

# visual
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt

# visual 3D
from mpl_toolkits import mplot3d

# notebook
from IPython.display import display, HTML


# torch
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import Sequential
from torch.nn import Sigmoid,ReLU
from torch.nn import Embedding,Linear,Dropout
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor,Compose
from torch.optim import SparseAdam,Adam,Adagrad,SGD

# colab
# from google.colab import drive  

**Hide Warnings**

In [ ]:
warnings.filterwarnings('ignore')

**Disable Autoscrolling**

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};

<br><br><br>
<br><br><br>
<br><br><br>

## Question 1: Dataset Preparation (Ingestion)

---


<br><br>

This implementation contains one file for training and two files for testing:   
- ml-1m.train.rating   
- ml-1m.test.rating  
- ml-1m.test.negative   

<br>
(feel free to use visual explanations)
<br>

a. **Explain** the role and structure of each file and how it was created from the original <a href='https://github.com/hexiangnan/neural_collaborative_filtering/tree/master/Data'>MovieLens 1M rating dataset</a>.

b. **Explain** how the training dataset is created.

- We iterate through each line of the train file.
- We find the number of items and users.
- We create a sparse matrix with NXM (users X items)
- We go through the file again and for each user item pair in the file line we check if there has been any positive rating.
- If there has been a positive rating we add a binary indicator of implicit feedback (1).

c. **Explain** how the test dataset is created.

Test dataset has been created in a similar manner to the train dataset, except for the negative samples which have been created as vector.
Each line contains 99 negative samples(the id of the user matches the index of the line).

#### Data Preperations:

In [5]:
# urls
train_url = "https://github.com/hexiangnan/neural_collaborative_filtering/blob/master/Data/ml-1m.train.rating?raw=true"
test_url = "https://github.com/hexiangnan/neural_collaborative_filtering/blob/master/Data/ml-1m.test.rating?raw=true"
test_neg_url = "https://github.com/hexiangnan/neural_collaborative_filtering/blob/master/Data/ml-1m.test.negative?raw=true"


In [9]:
import requests

def get_num_users_and_items(data_lines):
    num_users = 0
    num_items = 0
    for line in data_lines:
        line_values = line.split("\t")
        user_id = int(line_values[0])
        item_id = int(line_values[1])
        num_users = max(num_users, user_id)
        num_items = max(num_items, item_id)

    return num_users, num_items

def generate_matrix(data_lines, num_users, num_items):
    matrix = sp.dok_matrix((num_users + 1, num_items + 1), dtype=np.float32)
    for line in data_lines:
        line_values = line.split("\t")
        user_id = int(line_values[0])
        item_id = int(line_values[1])
        rating = float(line_values[2])
        if rating > 0:
            matrix[user_id, item_id] = 1.0
    
    return matrix
    

def load_data_as_matrix(url):
    # download data with requests
    response = requests.get(url)
    # Read as a text file
    data = response.text
    # Split by lines
    data_lines = data.splitlines()
    # Get number of users and items
    num_users, num_items = get_num_users_and_items(data_lines)\
    # Construct matrix
    mat = generate_matrix(data_lines, num_users, num_items)
    
    return mat

def load_negatives_vector(url):
    # download data with requests
    response = requests.get(url)
    # Read as a text file
    data = response.text
    # Split by lines
    data_lines = data.splitlines()
    # Construct vector
    vector = []
    for line in data_lines:
        line_values = line.split("\t")
        vector.append([int(x) for x in line_values[1:]])
    
    return vector
     


In [10]:
train_matrix = load_data_as_matrix(train_url)
test_matrix = load_data_as_matrix(test_url)
test_neg_vector = load_negatives_vector(test_neg_url)

num_users, num_items = train_matrix.shape

<br><br><br>
<br><br><br>
<br><br><br>

## Question 2: Neural Collaborative Filtering 
<br><br>

## a. Build the following four models using the neural collaborative filtering approach: 
- Matrix Factorization (MF)
- Multi layer perceptron (MLP)
- Generalized Matrix Factorization (GMF) 
- NeuroMatrixFactorization (NMF)

**For each model, use the best hyper-parameters suggested in the neuMF paper.**

In [11]:
# Enum of the different models
class ModelType:
    MF = 0
    MLP = 1
    GMF = 2
    NeuMF = 3

In [12]:
# Define the model
class NCF(nn.Module):
    def __init__(self, num_users, num_items, embedding_size=10, mlp_layers_sizes=[10, 10], dropout=0.1, model_type=ModelType.NeuMF):
        super(NCF, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.emb_size = embedding_size
        self.mlp_layers_sizes = mlp_layers_sizes
        self.dropout = dropout

        if model_type != ModelType.MLP:
            self.emb_user_MF = nn.Embedding(num_embeddings=num_users, embedding_dim=1)
            self.emb_item_MF = nn.Embedding(num_embeddings=num_items, embedding_dim=1)
            self.gmf_layer = self._create_GMF_layer()

        if model_type != ModelType.MF and model_type != ModelType.GMF:
            self.MLP_layer = self._create_MLP_layer()
            self.emb_user_MLP = nn.Embedding(num_embeddings=num_users, embedding_dim=self.emb_size)
            self.emb_item_MLP = nn.Embedding(num_embeddings=num_items, embedding_dim=self.emb_size)
        
        self._init_weights()
        self.neu_mf = self._create_NeuMF_layer()

    def _init_weights(self):
        if self.emb_user_MLP is not None and self.emb_item_MLP is not None:
            nn.init.normal_(self.emb_user_MLP.weight, std=0.01)
            nn.init.normal_(self.emb_item_MLP.weight, std=0.01)
        if self.emb_user_MF is not None and self.emb_item_MF is not None:
            nn.init.normal_(self.emb_user_MF.weight, std=0.01)
            nn.init.normal_(self.emb_item_MF.weight, std=0.01)
    
    def _create_MLP_layer(self):
        layers = []
        num_layer = len(self.mlp_layers_sizes)
        for i in range(num_layer):
            input_size = self.emb_size if i == 0 else self.mlp_layers_sizes[i-1]
            layers.append(nn.Linear(input_size, self.mlp_layers_sizes[i]))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(p=self.dropout))
        return nn.Sequential(*layers)
    
    def _create_NEU_MF_layer(self):
        return nn.Sequential(
            nn.Linear(in_features=2, out_features=1),
            nn.Sigmoid()
        )
    
    def _create_GMF_layer(self):
        return nn.Sequential(
            nn.Linear(in_features=1, out_features=1),
            nn.ReLU()
        )
    
    def forward(self, users, items):
        # GMF + MF
        if self.model_type != ModelType.MLP:
            emb_user_MF = self.emb_user_MF(users)
            emb_item_MF = self.emb_item_MF(items)
            # IF it's only MF return UxI
            if self.model_type == ModelType.MF:
                return emb_user_MF @ emb_item_MF.T
            
            gmf_vector = emb_user_MF * emb_item_MF
            gmf_vector = self.gmf_layer(gmf_vector)
        
        # MLP
        if self.model_type != ModelType.GMF and self.model_type != ModelType.MF:
            emb_user_MLP = self.emb_user_MLP(users)
            emb_item_MLP = self.emb_item_MLP(items)
            mlp_vector = torch.cat([emb_user_MLP, emb_item_MLP], dim=-1)
            mlp_vector = self.MLP_layer(mlp_vector)

        if self.model_type == ModelType.NeuMF:
            vector = torch.cat([mlp_vector, gmf_vector], dim=-1)
        elif self.model_type == ModelType.GMF:
            vector = gmf_vector
        else:
            vector = mlp_vector
        
        # NeuMF
        vector = self.neu_mf(vector)
        return vector 

<br><br><br><br>
#### Matrix Factorization (MF)  
<br>

In [ ]:
model_MF = NCF(num_users, num_items, 1, [], 0, ModelType.MF)

Model's architecture:

In [ ]:
# display/print the model architecture
print(model_MF)

<br><br><br><br><br><br>
#### Multi Layer Perceptron (MLP)

In [ ]:
model_MLP = NCF(num_users, num_items, 1, [10, 10], 0.1, ModelType.MLP)

Model's architecture:

In [ ]:
# display/print the model architecture
print(model_MLP)

<br><br><br><br><br><br>
####Generalized Matrix Factorization (GMF)

In [ ]:
model_GMF = NCF(num_users, num_items, 1, [], 0, ModelType.GMF)

Model's architecture:

In [ ]:
# display/print the model architecture
print(model_GMF)

<br><br><br><br><br><br>
#### NeuroMatrixFactorization (NMF)


In [ ]:
model_NMF = NCF(num_users, num_items, 1, [10, 10], 0.1, ModelType.NeuMF)

Model's architecture:

In [ ]:
# display/print the model architecture
print(model_NMF)

<br><br><br><br><br><br>

## b. Train and evaluate the recommendations accuracy of the models: 
- MF
- GMF
- MLP
- NMF

Compare the `LogLoss` and recommendations accuracy using `NDCG` and `MRR` metrics with cutoff values of 5 and 10.   
Discuss the comparison. 

**Metrics:**
- HitRatio
- nDCG
- MRR

In [ ]:
# Use your own metrics implementation OR use external packages for the metrics.
# If you are using external packages make sure they work properly. 
# A lot of the packages available does not work as you would expect.

**Evaluation:**

**HyperParams:**

In [ ]:
# the choosen hyperparams will effect your models & your grade 

<br><br>
Create train data:

<br><br>
train & eval:
- Create a training function 
- Evaluate the models trained and save the results accordingly 

In [ ]:
# feel free to change the function signature
def model_train(model):
    pass




<br><br><br><br>
<br><br><br><br>
<br><br><br><br>
All Results:

In [ ]:
# df_results
# each hyperparam will add a column to the dataframe
# this is an example for a df that would allow you to create plots easily

<br><br><br><br>
**Train & Validation Loss:**

Make sure you did not overfit.  
In case you did, fix that by adding early-stopping, regularization, etc.

In [ ]:
# plot
# make sure that you did not overfit!

**Training Time:**

In [ ]:
# plot

**Metric Evaluation:**

In [ ]:
# plot suggestion/example - you may create your own plot (you should achieve higher results)

<br><br><br><br>
<br><br><br><br>

**c. How do the values of MRR and NDCG differ between your current model and the results you got in the previous exercises which implemented the explicit recommendation approach? What are the differences in preparing the dataset for evaluation?**

**d. How will you measure item similarity using the NeuMF model?**

<br><br><br>
<br><br><br>
<br><br><br>

## Question 3: Loss Function 
<br><br>

#### a. One of the enhancements presented in the Neural Collaborative Filtering paper is the usage of probabilistic activation function (the sigmoid) and binary cross entropy loss function.    

Select one of the models you implemented in question 2 and change the loss function to a `Mean Squared Error` and the activation function of the last layer to `RELU`.   

Train the model and evaluate it in a similar way to what you did in question 2. 
Compare the results and discuss.

<br><br><br><br>
<br><br><br><br>
NMFs Results:

In [ ]:
# example: df_results[df_results.model.str.startswith('NMF')]

<br><br><br>
<br><br><br>

Train & Validation Loss:

In [ ]:
# plot

<br><br><br>
Training Time:

In [ ]:
# plot

<br><br><br>
Metric Evaluation:

In [ ]:
# plot

<br><br><br><br>
**Conclusions:**

    - In
    - Your
    - Own
    - Words
    

<br><br>
<br><br>


Good Luck :)